In [16]:
! pip install -U git+https://github.com/IINemo/isanlp.git@dev

  Cloning https://github.com/IINemo/isanlp.git (to revision dev) to /tmp/pip-req-build-ceyjsig5
  Running command git clone -q https://github.com/IINemo/isanlp.git /tmp/pip-req-build-ceyjsig5
  Running command git checkout -b dev --track origin/dev
  Switched to a new branch 'dev'
  Branch dev set up to track remote branch dev from origin.
  Created wheel for isanlp: filename=isanlp-0.0.6-cp36-none-any.whl size=33540 sha256=9a0e87136493f2fdda8b48764459bf0560951a93cb8015b915b8127fb41aa5a5
  Stored in directory: /tmp/pip-ephem-wheel-cache-mmjml67p/wheels/c2/22/fa/c2380ddd40a8fe7980f3ead4d7cbad4889975055f3bea6c15a
Successfully built isanlp
  Found existing installation: isanlp 0.0.6
    Uninstalling isanlp-0.0.6:
      Successfully uninstalled isanlp-0.0.6


In [1]:
%%bash

cp ../src/isanlp_rst/annotation.py ~/.pyenv/versions/3.6.7/lib/python3.6/site-packages/isanlp/annotation_rst.py
cp ../src/isanlp_rst/annotation_rst_pb2.py ~/.pyenv/versions/3.6.7/lib/python3.6/site-packages/isanlp/annotation_rst_pb2.py
cp ../src/isanlp_rst/annotation_from_protobuf.py ~/.pyenv/versions/3.6.7/lib/python3.6/site-packages/isanlp/annotation_from_protobuf.py

In [2]:
SERVER0 = ''
SERVER1 = ''
SERVER2 = ''

In [5]:
from isanlp.processor_remote import ProcessorRemote
from isanlp.processor_syntaxnet_remote import ProcessorSyntaxNetRemote
from isanlp import PipelineCommon
from isanlp.ru.converter_mystem_to_ud import ConverterMystemToUd

address_morph = (SERVER0, 4333)
address_syntax = (SERVER0, 4343)
address_rst = (SERVER1, 3490)
address_rst = (SERVER2, 3343)

ppl = PipelineCommon([(ProcessorRemote(address_morph[0], address_morph[1], 'default'),
                 ['text'],
                 {'tokens': 'tokens',
                  'sentences': 'sentences',
                  'postag': 'mystem_postag',
                  'lemma': 'lemma'}),
                (ProcessorSyntaxNetRemote(address_syntax[0], address_syntax[1]),
                 ['tokens', 'sentences'],
                 {'syntax_dep_tree': 'syntax_dep_tree'}),
                (ConverterMystemToUd(),
                 ['mystem_postag'],
                 {'morph': 'morph',
                  'postag': 'postag'}),
                (ProcessorRemote(address_rst[0], address_rst[1], 'default'),
                 ['text', 'tokens', 'sentences', 'postag', 'morph', 'lemma', 'syntax_dep_tree'],
                 {'rst': 'rst'})])

In [17]:
res = ppl('Вообще, парадокс - это выражение, в котором вывод не совпадает с посылкой и не вытекает из нее.')

In [18]:
res

{'text': 'Вообще, парадокс - это выражение, в котором вывод не совпадает с посылкой и не вытекает из нее.',
 'tokens': [<isanlp.annotation.Token at 0x7f4f055ee128>,
 'sentences': [<isanlp.annotation.Sentence at 0x7f4f055815f8>],
 'mystem_postag': [['ADV,вводн=',
   '',
   'S,муж,неод=(вин,ед|им,ед)',
   '',
   'PART=',
   'S,сред,неод=(вин,ед|им,ед)',
   '',
   'PR=',
   'APRO=(пр,ед,муж|пр,ед,сред)',
   'S,муж,неод=(вин,ед|им,ед)',
   'PART=',
   'V,нп=непрош,ед,изъяв,3-л,несов',
   'PR=',
   'S,жен,неод=твор,ед',
   'CONJ=',
   'PART=',
   'V,нп=непрош,ед,изъяв,3-л,несов',
   'PR=',
   'SPRO,ед,3-л,жен=(вин|род)',
   '']],
 'lemma': [['вообще',
   ',',
   'парадокс',
   '-',
   'это',
   'выражение',
   ',',
   'в',
   'который',
   'вывод',
   'не',
   'совпадать',
   'с',
   'посылка',
   'и',
   'не',
   'вытекать',
   'из',
   'она',
   '.']],
 'syntax_dep_tree': [[<isanlp.annotation.WordSynt at 0x7f4f0557aba8>,
 'morph': [[{'fPOS': 'ADV'},
   {},
   {'fPOS': 'NOUN',
    'Gender'

In [19]:
print(res['rst'])

In [20]:
def extr_pairs(tree):
    pp = []
    
    if tree.left:
        pp.append([tree.left.text, tree.right.text, tree.relation])
        pp += extr_pairs(tree.left)
        pp += extr_pairs(tree.right)
    return pp

In [21]:
print(res['rst'][0])

id: 12
text: Вообще, парадокс - это выражение, в котором вывод не совпадает с посылкой и не вытекает из нее.
relation: root
left: Вообще, парадокс - это выражение, в котором вывод не совпадает с посылкой и не вытекает из нее
ight: .



In [22]:
extr_pairs(res['rst'][0])

[['Вообще, парадокс - это выражение, в котором вывод не совпадает с посылкой и не вытекает из нее',
  '.',
  'root'],
 ['Вообще, парадокс - это выражение, в котором вывод не совпадает с посылкой и',
  ' не вытекает из нее',
  'relation'],
 ['Вообще, парадокс - это выражение, в котором вывод не совпадает с посылкой',
  ' и',
  'relation'],
 ['Вообще, парадокс',
  ' - это выражение, в котором вывод не совпадает с посылкой',
  'relation'],
 [' - это выражение', ', в котором вывод не совпадает с посылкой', 'relation'],
 [',', ' в котором вывод не совпадает с посылкой', 'relation']]